<a href="https://colab.research.google.com/github/prasadsurapaneni/prasadsurapaneni/blob/main/ndap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install tesseract-ocr
!tesseract --version
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils

In [ ]:
!pip3 install camelot-py[cv] tabula-py
!apt install ghostscript python3-tk

In [7]:
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import pytesseract
import sys
from pdf2image import convert_from_path
import os
from IPython.display import Image

import pandas as pd
import camelot
import tabula
import numpy as np
import math

pytesseract.get_languages()

['eng', 'osd']

In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
!wget --header="Host: doc-0s-bs-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_6q6v5cpktfsbr56mgd97qaku8fu7piaj_nonce=kuic01bt82ar6" --header="Connection: keep-alive" "https://doc-0s-bs-docs.googleusercontent.com/docs/securesc/nka8he3o66l2nqa1l22al91eq1deois1/6c75ltgba1vf8i2hvtq8u93s5o25bb0o/1635751725000/06457702143635460075/06457702143635460075/1SnWUjlKo5mBD8fd601xry8USDi-dLgqi?e=download&authuser=0&nonce=kuic01bt82ar6&user=06457702143635460075&hash=u0q539qh0be37843qeaatc2n1i6t6gg0" -c -O 'Report_585_75th_round_Education_final_1507_0.pdf'

--2021-11-01 07:30:23--  https://doc-0s-bs-docs.googleusercontent.com/docs/securesc/nka8he3o66l2nqa1l22al91eq1deois1/6c75ltgba1vf8i2hvtq8u93s5o25bb0o/1635751725000/06457702143635460075/06457702143635460075/1SnWUjlKo5mBD8fd601xry8USDi-dLgqi?e=download&authuser=0&nonce=kuic01bt82ar6&user=06457702143635460075&hash=u0q539qh0be37843qeaatc2n1i6t6gg0
Resolving doc-0s-bs-docs.googleusercontent.com (doc-0s-bs-docs.googleusercontent.com)... 142.251.45.1, 2607:f8b0:4004:83e::2001
Connecting to doc-0s-bs-docs.googleusercontent.com (doc-0s-bs-docs.googleusercontent.com)|142.251.45.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59621419 (57M) [application/pdf]
Saving to: ‘Report_585_75th_round_Education_final_1507_0.pdf’

Report_585_75th_rou 100%[===================>]  56.86M  61.1MB/s    in 0.9s    

2021-11-01 07:30:24 (61.1 MB/s) - ‘Report_585_75th_round_Education_final_1507_0.pdf’ saved [59621419/59621419]



In [2]:
pdffile = "/content/5.List of Products Identified under ODOP Initiative (Invest India).pdf"

# pages = convert_from_path(pdffile, first_page=1429, last_page=1429)

# for idx, page in enumerate(pages):
#     filename = "page_"+str(idx+1)+".jpg"
#     page.save(filename, 'JPEG')

# outfile = "outfile.txt"
# f = open(outfile, "a")
# img_file = "140.SSC.jpg"
# cv2_imshow(img_file)
# text = str(pytesseract.image_to_string(Image.open(img_file), config=r'--oem 3 --psm 6'))
# text = str(pytesseract.image_to_string(cv2.imread(img_file)))

# print(text)

In [ ]:
img_file = "page_1.jpg"
# cv2_imshow(img_file)
Image(img_file)

# Reading tables from PDF using OCR techniques

In [ ]:
def find_tables(image):
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
        ~blurred,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))
    
    mask = horizontally_dilated + vertically_dilated
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE,
    )

    MIN_TABLE_AREA = 1e5
    contours = [c for c in contours if cv2.contourArea(c) > MIN_TABLE_AREA]
    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.1 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

    # The link where a lot of this code was borrowed from recommends an
    # additional step to check the number of "joints" inside this bounding rectangle.
    # A table should have a lot of intersections. We might have a rectangular image
    # here though which would only have 4 intersections, 1 at each corner.
    # Leaving that step as a future TODO if it is ever necessary.
    images = [image[y:y+h, x:x+w] for x, y, w, h in bounding_rects]
    return images

In [ ]:
image_filename = "page_1.jpg"
image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
image = find_tables(image)[0]
cv2.imwrite("page_table.png", image)

True

In [ ]:
Image("page_table.png")

In [ ]:
BLUR_KERNEL_SIZE = (17, 17)
STD_DEV_X_DIRECTION = 0
STD_DEV_Y_DIRECTION = 0
blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)

In [ ]:
# image = ~cv2.imread("page_table1.png", cv2.IMREAD_GRAYSCALE)
cv2.imwrite("page_table_blurred.png", blurred)

True

In [ ]:
Image("page_table_blurred.png")

In [ ]:
MAX_COLOR_VAL = 255
BLOCK_SIZE = 15
SUBTRACT_FROM_MEAN = -2

img_bin = cv2.adaptiveThreshold(
    ~blurred,
    MAX_COLOR_VAL,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    BLOCK_SIZE,
    SUBTRACT_FROM_MEAN,
)

cv2.imwrite("page_table_thresholded.png", img_bin)

True

In [ ]:
Image("page_table_thresholded.png")

In [ ]:
def extract_cell_images_from_table(image):
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
        ~blurred,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))
    
    mask = horizontally_dilated + vertically_dilated
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
    )
    
    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.05 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    
    # Filter out contours that aren't rectangular. Those that aren't rectangular
    # are probably noise.
    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]
    
    # Filter out rectangles that are too narrow or too short.
    MIN_RECT_WIDTH = 40
    MIN_RECT_HEIGHT = 10
    bounding_rects = [
        r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
    ]
    
    # The largest bounding rectangle is assumed to be the entire table.
    # Remove it from the list. We don't want to accidentally try to OCR
    # the entire table.
    largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
    bounding_rects = [b for b in bounding_rects if b is not largest_rect]
    
    cells = [c for c in bounding_rects]
    def cell_in_same_row(c1, c2):
        c1_center = c1[1] + c1[3] - c1[3] / 2
        c2_bottom = c2[1] + c2[3]
        c2_top = c2[1]
        return c2_top < c1_center < c2_bottom
    
    orig_cells = [c for c in cells]
    rows = []
    while cells:
        first = cells[0]
        rest = cells[1:]
        cells_in_same_row = sorted(
            [
                c for c in rest
                if cell_in_same_row(c, first)
            ],
            key=lambda c: c[0]
        )
    
        row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
        rows.append(row_cells)
        cells = [
            c for c in rest
            if not cell_in_same_row(c, first)
        ]
    
    # Sort rows by average height of their center.
    def avg_height_of_center(row):
        centers = [y + h - h / 2 for x, y, w, h in row]
        return sum(centers) / len(centers)
    
    rows.sort(key=avg_height_of_center)
    cell_images_rows = []
    for row in rows:
        cell_images_row = []
        for x, y, w, h in row:
            cell_images_row.append(image[y:y+h, x:x+w])
        cell_images_rows.append(cell_images_row)
    return cell_images_rows

In [ ]:
image = cv2.imread("page_table.png", cv2.IMREAD_GRAYSCALE)
cell_images_rows = extract_cell_images_from_table(image)
cv2.imwrite("page_table_cell-1-1.png", cell_images_rows[4][4])

True

In [ ]:
Image("page_table_cell-1-1.png")

In [ ]:
def crop_to_text(image):
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2

    img_bin = cv2.adaptiveThreshold(
        ~image,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )

    img_h, img_w = image.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
    horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    both = horizontal_lines + vertical_lines
    cleaned = img_bin - both

    # Get rid of little noise.
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    opened = cv2.dilate(opened, kernel)

    contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    bounding_rects = [cv2.boundingRect(c) for c in contours]
    NUM_PX_COMMA = 6
    MIN_CHAR_AREA = 5 * 9
    char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
    if char_sized_bounding_rects:
        minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
        for x, y, w, h in char_sized_bounding_rects:
            minx = min(minx, x)
            miny = min(miny, y)
            maxx = max(maxx, x + w)
            maxy = max(maxy, y + h)
        x, y, w, h = minx, miny, maxx - minx, maxy - miny
        cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
    else:
        # If we morphed out all of the text, assume an empty image.
        cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
    bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
    return bordered

In [ ]:
image = cv2.imread("page_table_cell-1-1.png", cv2.IMREAD_GRAYSCALE)
image = crop_to_text(image)
cv2.imwrite("page_table_cell-1-1_cropped.png", image)

True

In [ ]:
Image("page_table_cell-1-1_cropped.png")

# Extracting specific pages from PDF and create new PDF

In [ ]:
!pip install PyPDF2

In [2]:
# pages_required = ({'start':56,'end':59}, {'start':102,'end':325}, {'start':328,'end':328},)
pages_required = ({'start':7,'end':15},)

In [4]:
from PyPDF2 import PdfFileReader, PdfFileWriter

# Note: index starts at 1 and is inclusive of the end. 
# The following will extract page 3 of the pdf file.
pdfs = {'/content/Arunachal_Pradesh.pdf': pages_required}  

# print(pdfs.items())

for pdf, segments in pdfs.items():
    pdf_reader = PdfFileReader(open(pdf, 'rb'))
    pdf_writer = PdfFileWriter()
    for segment in segments:
        start_page = segment['start']
        end_page = segment['end']
        for page_num in range(start_page - 1, end_page):
            pdf_writer.addPage(pdf_reader.getPage(page_num))
    output_filename = f'{pdf}_required_pages_to_extract.pdf'
    with open(output_filename,'wb') as out:
        pdf_writer.write(out)

In [6]:
# split each page into a separate pdf
from PyPDF2 import PdfFileReader, PdfFileWriter

# Note: index starts at 1 and is inclusive of the end. 
# The following will extract page 3 of the pdf file.
inputpdf = PdfFileReader(open("/content/Report_585_75th_round_Education_final_1507_0.pdf_required_pages_to_extract.pdf", "rb")) 

# print(inputpdf.numPages)

for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("NSSEDU75/NSS_Edu_page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)


In [9]:
import shutil
shutil.make_archive("NSSEDU", 'zip', "/content/NSSEDU75") 

'/content/NSSEDU.zip'

# Reading tables from pdf with camelot

In [8]:
tables = camelot.read_pdf(pdffile, pages='all', flavour='lattice')

In [9]:
print(tables)
print(tables[0].parsing_report)

<TableList n=16>
{'accuracy': 99.62, 'whitespace': 1.67, 'order': 1, 'page': 1}


In [ ]:
tables[0].df

In [44]:
final_df = pd.DataFrame()
for i in range(len(tables)):
    # print(tables[i])
    # final_df = final_df.reindex(columns=tables[i].columns, fill_value=0)
    final_df = pd.concat([final_df, tables[i].df], ignore_index=True)

final_df = final_df.iloc[1:]
final_df.columns = final_df.iloc[0]
final_df.drop(['Sr. No'], axis=1, inplace=True)
final_df = final_df[1:].reset_index(drop=True)
final_df = final_df.replace(r'\s+|\\n', ' ', regex=True)
print(final_df)

1                   State Name  ...                        Product Name
0    Andaman & Nicobar Islands  ...                                Tuna
1    Andaman & Nicobar Islands  ...  Coconut and coconut based products
2    Andaman & Nicobar Islands  ...                           Fisheries
3               Andhra Pradesh  ...                  Readymade Garments
4               Andhra Pradesh  ...              Srikalahasti Kalamkari
..                         ...  ...                                 ...
785                West Bengal  ...                     Wooden Products
786                West Bengal  ...                     Wooden Products
787                West Bengal  ...          IT and IT enables services
788                West Bengal  ...                             Tourism
789                West Bengal  ...                  Bardhaman Mihidana

[790 rows x 3 columns]


In [45]:
final_df.to_csv("ODOP_InvestIndia.csv", index=False)

In [ ]:
tables.export('page_1429.csv', f='csv', compress=True) # json, excel, html, markdown, sqlite

In [ ]:
!unzip "page_1429.zip"

Archive:  page_1429.zip
 extracting: page_1429-page-68-table-1.csv  


In [ ]:
tables[0]

<Table shape=(26, 19)>

In [ ]:
tables[0].to_excel('page_1429.xlsx')

# Reading tables from pdf with tabula

In [31]:
# pdffile = "/content/Report_585_75th_round_Education_final_1507_0.pdf_required_pages_to_extract.pdf"
tables = tabula.read_pdf(pdffile, pages='all', lattice=False)  

In [ ]:
tabula.convert_into(pdffile, "pdf_excels.csv", output_format='csv', pages='1,2,3,4', stream=True)

In [32]:
tables

[                                           Unnamed: 0   Unnamed: 1     NFHS-3
 0                         Indicators NFHS-4 (2015-16)          NaN  (2005-06)
 1               Population and Household ProfileUrban  Rural Total      Total
 2   1. Population (female) age 6 years and above w...    78.5 80.9       77.1
 3           2. Population below age 15 years (%) 25.0    21.8 25.0       28.7
 4   3. Sex ratio of the total population (females ...      839 854        814
 5   4. Sex ratio at birth for children born in the...        * 812        840
 6   5. Children under age 5 years whose birth was ...  (89.5) 88.0       62.4
 7             6. Households with electricity (%) 99.8   100.0 99.8       99.3
 8   7. Households with an improved drinking-water ...    69.7 80.0       90.5
 9   8. Households using improved sanitation facili...    87.7 73.3       62.6
 10  9. Households using clean fuel for cooking3 (%...    80.8 97.9       79.6
 11         10. Households using iodized salt (%) 98

In [33]:
for table in tables:
    print(table)
    # table.to_csv('page_1429_tabula.csv', index=False) # json, excel, html, markdown, sqlite

                                           Unnamed: 0   Unnamed: 1     NFHS-3
0                         Indicators NFHS-4 (2015-16)          NaN  (2005-06)
1               Population and Household ProfileUrban  Rural Total      Total
2   1. Population (female) age 6 years and above w...    78.5 80.9       77.1
3           2. Population below age 15 years (%) 25.0    21.8 25.0       28.7
4   3. Sex ratio of the total population (females ...      839 854        814
5   4. Sex ratio at birth for children born in the...        * 812        840
6   5. Children under age 5 years whose birth was ...  (89.5) 88.0       62.4
7             6. Households with electricity (%) 99.8   100.0 99.8       99.3
8   7. Households with an improved drinking-water ...    69.7 80.0       90.5
9   8. Households using improved sanitation facili...    87.7 73.3       62.6
10  9. Households using clean fuel for cooking3 (%...    80.8 97.9       79.6
11         10. Households using iodized salt (%) 98.5    99.5 98

In [23]:
type(tables[0])

pandas.core.frame.DataFrame

In [30]:
df = pd.DataFrame(tables[0])

df.head()

,Unnamed: 0,Unnamed: 1,NFHS-3
0,Indicators NFHS-4 (2015-16),NaN,(2005-06)
1,Population and Household ProfileUrban,Rural Total,Total
2,1. Population (female) age 6 years and above w...,78.5 80.9,77.1
3,2. Population below age 15 years (%) 25.0,21.8 25.0,28.7
4,3. Sex ratio of the total population (females ...,839 854,814


# Converting PDF to Excel/CSV with PDFtables API

In [ ]:
pip install --upgrade git+https://github.com/pdftables/python-pdftables-api.git

In [ ]:
import pdftables_api

pdffile = "/content/nss_Report_570_1.pdf_required_pages_to_extract.pdf"
c = pdftables_api.Client('vmwtpp99obsk')
c.xlsx(pdffile, 'nss_household_tables') 

# Read unstructured excel files

In [ ]:
# !apt-get update
!pip install xlrd==1.2.0

In [2]:
import xlrd
xlrd.__VERSION__

'1.2.0'

In [ ]:
xl_file = "table_1_excel.xlsx"
df = pd.ExcelFile(xl_file)

print(df)

In [ ]:
# df.parse()

In [ ]:
df = pd.read_excel(xl_file)

In [ ]:
df

In [5]:
indpark_df = pd.read_csv(r"/content/6 Ind parks_Final.csv", encoding='cp1252') #, engine='xlrd'
indpark_df

,State,Year,Park ID,User,User Level,District,Park Name,Industrial Park Type,Total Land Area (ha.),Land Available (ha.),Plug And Play,Park Type,Sector,Update Date
0,VARCHAR(80),VARCHAR(20),VARCHAR(20),VARCHAR(50),VARCHAR(50),VARCHAR(80),VARCHAR(255),VARHAR(30),DECIMAL,DECIMAL,VARCHAR(20),VARCHAR(30),VARCHAR(50),DATE
1,MEGHALAYA,2022,6868,Meghalaya,State,Ribhoi,Export Promotion Industrial Park (EPIP),Industrial Park,125.45,0,No,Public,Mixed,2022-02-22
2,CHHATTISGARH,2022,3194,doicg_iis,State,Raipur,"Electronic Manufacturing Cluster, Naya Raipur",Industrial Area,48.57,22.84,No,Public,Electronics Hardware,2022-02-22
3,KERALA,2022,7853,keralasidco,State,Thrissur,"Kerala Sidco Mini Industrial Estate, Kattoor",Industrial Park,0,0.08,NaN,Public,Capital Goods,NaN
4,Odisha,2022,5944,Odisha,State,Khordha,"EMC Park, Infovalley, Bhubaneswar",Industrial Park,82.15,44.92,NaN,Public,Electronics Hardware,2022-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4554,GUJARAT,2022,421,Gujarat,State,Ahmadabad,Bhagapura I,Industrial Estate,285.49,100,Yes,Public,Mixed,2020-07-09
4555,KARNATAKA,2022,7097,KIADB,State,Bagalkot,Navanagar Agrotech Park,Industrial Area,89.59,0,NaN,Public,Mixed,2020-11-12
4556,Odisha,2022,5938,Odisha,State,Bolangir,I.E Bolangir Zone-A,Industrial Estate,5.45,0.28,NaN,Public,Mixed,NaN
4557,KARNATAKA,2022,7037,KIADB,State,Bangalore Rural,Bengaluru Hardware Park,Industrial Area,539,0,NaN,Public,Electronics Hardware,2021-03-04


In [7]:
indpark_df.to_csv("industrial parks.csv", encoding='utf-8', index=False)

# Reading Excel files using pandas

In [ ]:
workSheets = ['Page 125','Page 126','Page 127','Page 128','Page 129','Page 130','Page 131','Page 132','Page 133','Page 134','Page 135','Page 136','Page 137','Page 138','Page 139','Page 140','Page 141','Page 142','Page 143','Page 144','Page 145','Page 146','Page 147','Page 148','Page 149','Page 150','Page 151','Page 152','Page 153','Page 154','Page 155','Page 156','Page 157','Page 158','Page 159','Page 160','Page 161','Page 162','Page 163','Page 164','Page 165','Page 166','Page 167','Page 168','Page 169','Page 170','Page 171','Page 172','Page 173','Page 174','Page 175','Page 176','Page 177','Page 178','Page 179','Page 180','Page 181','Page 182','Page 183','Page 184','Page 185','Page 186','Page 187','Page 188','Page 189','Page 190','Page 191','Page 192','Page 193','Page 194','Page 195','Page 196','Page 197','Page 198']
skipRows = 6
excelFile = "/content/Table -7.xlsx"

In [ ]:
df = {}
for ws in workSheets:
    df[ws] = pd.read_excel(excelFile, sheet_name=ws, header=None)

In [ ]:
# for ws in workSheets:
    #  print(df[ws].loc[:, df[ws].isna().any()])
#     print(df[ws])
#     break

# (df['Page 48'].drop(1, axis=1, inplace=True))
(df['Page 125'])

In [ ]:
sets1 = 9
sets2 = 3
sets3 = 2
columnsToAdd = {}

# columnsToAdd['AgeGroup'] = list(np.repeat(['5-29 years', '3-35 yeas'],sets1)) #, 'rural+urban'
columnsToAdd['Sector'] = list(np.repeat(['rural', 'urban', 'rural+urban'],sets2)) #, 'rural+urban'
# columnsToAdd['Gender'] = list(np.repeat(['male', 'female', 'person'],sets2))*sets2
columnsToAdd['Gender'] = ['male', 'female', 'person']*sets2
# columnsToAdd['Typeofresidence'] = ['students hostel', 'others', 'all']*sets1

In [ ]:
print(columnsToAdd.items())

dict_items([('Sector', ['rural', 'rural', 'rural', 'urban', 'urban', 'urban', 'rural+urban', 'rural+urban', 'rural+urban']), ('Gender', ['male', 'female', 'person', 'male', 'female', 'person', 'male', 'female', 'person'])])


In [ ]:
for i, ws in enumerate(workSheets):
    # df[ws] = df[ws][6:]
    df[ws].dropna(inplace=True)
    for k, v in columnsToAdd.items():
        df[ws][k] = v[i]
    
    df[ws].reset_index(drop=True, inplace=True)


In [ ]:
for ws in workSheets:
    df[ws] = df[ws].reset_index(drop=True)
    df[ws] = df[ws].T.reset_index(drop=True).T
    if df[ws][0][0]=="State":     
        df[ws].drop(0, inplace=True)
    df[ws] = df[ws].reset_index(drop=True)
    if df[ws][0][0]=="current place of\nresidence" or df[ws][0][0]=="current place ofresidence" or df[ws][0][0]=="residence":     
        df[ws].drop(0, inplace=True)
    df[ws] = df[ws].reset_index(drop=True)
    if df[ws][0][0]=="State/ UT.":     
        df[ws].drop(0, inplace=True)
    df[ws] = df[ws].reset_index(drop=True)
    if df[ws][0][0]=="(1)":
        df[ws].drop(0, inplace=True)
    df[ws] = df[ws].reset_index(drop=True)
    print(df[ws])
    # break

                                                   0      1   ...     19    20
0                                      Andhra Pradesh    100  ...  rural  male
1                                   Arunachal Pradesh   33.7  ...  rural  male
2                                               Assam   47.4  ...  rural  male
3                                               Bihar     31  ...  rural  male
4                                        Chhattisgarh   80.8  ...  rural  male
5                                               Delhi      0  ...  rural  male
6                                                 Goa      0  ...  rural  male
7                                             Gujarat    100  ...  rural  male
8                                             Haryana   89.4  ...  rural  male
9                                    Himachal Pradesh   84.3  ...  rural  male
10                                    Jammu & Kashmir    100  ...  rural  male
11                                          Jharkhan

In [ ]:
sheets = []
for ws in workSheets:
    sheets.append(df[ws])

pd.concat(sheets, ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Andhra Pradesh,100,0,0,100,1812,33,98.2,0.2,1.6,100,5911,121,99.6,0,0.4,100,1076,39,rural,male
1,Arunachal Pradesh,33.7,0,66.3,100,2,2,100,0,0,100,8,2,100,0,0,100,6,3,rural,male
2,Assam,47.4,39.3,13.3,100,252,15,45.8,38.1,16.1,100,1100,60,66.9,11.5,21.6,100,556,23,rural,male
3,Bihar,31,51.6,17.4,100,768,23,41.3,43.4,15.3,100,4854,113,54.5,40.1,5.4,100,2158,40,rural,male
4,Chhattisgarh,80.8,15.5,3.7,100,290,8,67.4,28.2,4.4,100,1141,42,84.5,15.5,0,100,278,14,rural,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Lakshadweep,52.1,26.7,21.2,100,7,12,100,0,0,100,2,4,100,0,0,100,0,1,rural+urban,person
347,Puducherry,100,0,0,100,52,11,100,0,0,100,216,35,100,0,0,100,71,16,rural+urban,person
348,all-India,83.3,10.1,6.6,100,87599,3356,85.3,9.2,5.5,100,"3,00,415",11876,87.5,8.6,3.6,100,"1,32,532",5952,rural+urban,person
349,estimated number of students in private unaide...,72976,8855,5768,87599,xxxx,xxxx,"2,56,359",27491,16517,"3,00,415",xxxx,xxxx,"1,16,006",11375,4730,"1,32,532",xxxx,xxxx,rural+urban,person


In [ ]:
# df['Page 10'].drop(0, inplace=True)
# df['Page 10'].drop(1, inplace=True)
# df['Page 12'].drop(0, inplace=True)
# df['Page 12'].drop(1, inplace=True)
# df['Page 2']
# df['Page 3']
pd.concat(sheets, ignore_index=True).to_excel('Table A48.xlsx')

# Dataframe Transformations

In [ ]:
df = pd.read_excel("/content/120_Quater_wise(3-6years)_2017_2018.xlsx")

In [ ]:
df

In [ ]:
df1 = pd.melt(df, id_vars=['State', 'Quarter', 'Month', 'Year'], var_name = 'Gender') \
        # .pivot_table(index=['Category', 'Campus'], columns='Year', values='value') \
        # .reset_index()

In [ ]:
df1

In [ ]:
df1.to_excel("120_Quater_wise(3-6years)_2017_2018.xlsx")

In [ ]:
df1[['TypeofSchool','Distance1']] = df1['Distance'].str.split('_', expand=True)

In [ ]:
df1['Distance'] = df1['Distance1']
df1.drop('Distance1', axis=1)

In [ ]:
df1.to_excel("5_Statement 4_3_1+4_5_1 Transformeddd.xlsx")

In [ ]:
df2 = pd.pivot_table(df1, index=['State/UT', 'Sector','Distance'], columns= 'TypeofSchool', values='value').reset_index()

In [ ]:
df2
# df2.to_excel('3_2_1_transformed.xlsx')

# Tranformation Handbook - Infrastructure

In [ ]:
from functools import reduce

In [ ]:
path = 'infra'
files = sorted([f for f in os.listdir(path) if not f.startswith('.') and f!='130T_24112151FD3DD4EC3649F5BA15BB925829569E.XLSX'])
print(files)

['122T_241121A3B9148F92634ECBB28DCC8879FB37EA.XLSX', '123T_241121ABF25A0AAA374265892D10A8AFBD1B4C.XLSX', '124T_24112148E624F65FC84E338C5162E441F0BA98.XLSX', '125T_2411210DCBBC82CF134511A47C41056C829E51.XLSX', '126T_2411214111A440F9F742C19D77D0B59BF0B624.XLSX', '127T_24112123EC2D69B15A45ECACC94849D2CE72A3.XLSX', '128T_241121A77CB75620064029A3BEFE86623DDFE8.XLSX', '129T_24112198C585166D454427A58ADCF7801DAF49.XLSX', '131T_2411217DA7733EB8814AEAB461E41C2894BC48.XLSX', '132T_241121C13C9E251C3944FF844DF4F5169D32CA.XLSX']


In [ ]:
ds = {}
tdf = {}
skip_rows = 3
key_col = 'State/Union Territory'
for i, file in enumerate(files):
    xls = pd.ExcelFile(os.path.join(path,file))
    # print(file)
    tmpLst = []
    for sheet in xls.sheet_names:
        tdf[sheet] = pd.read_excel(xls, sheet_name=sheet)
        if (xls.sheet_names[0]==sheet):
            # print(tdf[sheet].iloc[0])
            title = tdf[sheet].iloc[0]['Unnamed: 1'].split(':')[1].strip()
            title_ext = tdf[sheet].iloc[1]['Unnamed: 1'].strip()
            title = title+" "+title_ext
        tdf[sheet] = pd.read_excel(xls, sheet_name=sheet, skiprows=skip_rows)
        # tdf[sheet] = tdf[sheet][(skip_rows-1):]
        tdf[sheet].drop('Unnamed: 0', axis=1, inplace=True)
        # tdf[sheet].columns = tdf[sheet].iloc[0]
        # tdf[sheet] = tdf[sheet][1:].reset_index(drop=True)
        tdf[sheet].replace("-","", inplace=True)
        tdf[sheet].replace(".","", inplace=True)
        # print(tdf[sheet])
        # print(tdf[sheet].columns)
        if (xls.sheet_names[-1]==sheet):
            n_rows = tdf[sheet].loc[tdf[sheet][key_col] == "ALL INDIA"].index[0]
            tdf[sheet] = tdf[sheet][:(n_rows+1)]
        tmpLst.append(tdf[sheet])
    ds[i] = reduce(lambda  left,right: pd.merge(left,right,on=[key_col],
                                            how='outer'), tmpLst)
    # print(ds[i].columns)
    for column_name in ds[i].columns[1:]:
        column_name = str(column_name)
        ds[i].rename(columns={column_name : (column_name[:2]+column_name[-2:])}, inplace=True)
    
    #transform dataset
    ds[i] = pd.melt(ds[i], id_vars=['State/Union Territory'], var_name = 'Year', value_name=title)
    ds[i]['Year'] = ds[i]['Year'].astype('string')
    # print(ds[i])
    # break


In [ ]:
file_name = '/content/infra/130T_24112151FD3DD4EC3649F5BA15BB925829569E.XLSX'
df130 = pd.read_excel(file_name)

title = df130.iloc[0]['Unnamed: 1'].split(':')[1].strip()
df130 = df130[1:-5]
df130.drop('Unnamed: 0', axis=1, inplace=True)
df130.columns = df130.iloc[0]
df130 = df130[1:].reset_index(drop=True)
df130.replace("-","", inplace=True)
df130.replace(".","", inplace=True)

df130 = pd.melt(df130, id_vars=['State/Union Territory'], var_name = 'Year', value_name=title)
df130[['Year','Category']] = df130['Year'].str.split("_", 1, expand=True)
# df130 = df130.set_index(['State/Union Territory', 'Year']).unstack('Category')
df130 = pd.DataFrame(pd.pivot_table(df130, index=['State/Union Territory', 'Year'], columns=['Category'], values=title, aggfunc='first').to_records())
df130 = df130.fillna('')
# print(title)
print(df130)

           State/Union Territory  ... STATE-WISE EASE OF DOING BUSINESS SCORE
0    Andaman and Nicobar Islands  ...                                    9.73
1    Andaman and Nicobar Islands  ...                                     0.3
2    Andaman and Nicobar Islands  ...                                    1.25
3    Andaman and Nicobar Islands  ...                                        
4                 Andhra Pradesh  ...                                   70.12
..                           ...  ...                                     ...
139                  Uttarakhand  ...                                        
140                  West Bengal  ...                                    46.9
141                  West Bengal  ...                                   84.23
142                  West Bengal  ...                                   94.59
143                  West Bengal  ...                                        

[144 rows x 4 columns]


In [ ]:
# df130.to_excel('handbook_infra1.xlsx')

In [ ]:
tf_dfs = [v for k,v in ds.items()]
tf_dfs.append(df130)
final_df = reduce(lambda  left,right: pd.merge(left,right,on=[key_col, 'Year'],
                                            how='outer'), tf_dfs)
final_df = final_df[final_df[key_col]!='ALL INDIA'].sort_values(['Year', key_col])
final_df.to_excel('handbook_infra_final.xlsx', index=False)

# Scraping with beautiful soup

In [6]:
!pip install html-table-parser-python3

In [11]:
url = "https://iis.ncog.gov.in/parks/exploreParkReport"

In [24]:
# using HTMLTableParser

import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd


def url_get_contents(url):
    # print(url)
    req = urllib.request.Request(url=url)
    # print(req)
    f = urllib.request.urlopen(req)
    
    return f.read()

xhtml = url_get_contents(url)
xhtml = xhtml.decode("utf-8")

print(xhtml)

# parserObj = HTMLTableParser()

# parserObj.feed(xhtml)

# pprint(parserObj.tables)

# df = pd.DataFrame(parserObj.tables[0])

# df

HTTPError: ignored

In [ ]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

In [35]:
# https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup
# using beautifulsoup 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = url

page = requests.get(url)

# print(page.content)

soup = BeautifulSoup(page.text, 'lxml')
soup

<html>
<head><title>India Industrial Land Bank</title>
</head>
<body onload="myFunction();">
<!-- That's an error!  -->
</body>
<script>
function myFunction() {
  setInterval(function(){ 
//window.location.href = "https://iis.ncog.gov.in/parks"; }, 1);
}
</script>
<!-- <center>
        <h1>Sorry, the page you requesting is not available</h1>
		<a href="https://iis.ncog.gov.in/parks">Home Page</a>
    </center> -->
</html>

In [14]:
table = soup.find('table', attrs={'class':'datatable'})

table

headers = []
for i in table.find_all('th'):
    label = i.text
    headers.append(label.strip("\n"))

df1 = pd.DataFrame(columns = headers)
df1.columns

AttributeError: ignored

In [58]:
for j in table.find_all('tr'):
    # print(j, "\n\n\n")
    row_data = j.find_all('td')
    # print(len(row_data))
    # print("\n")
    row = [re.sub("\n\r\n\s*|\n","",i.text) for i in row_data]
    # print(row)
    length = len(df1)
    if(len(df1.columns) == len(row)):
        df1.loc[length] = row

In [ ]:
df1

# Scraping with Selenium

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [6]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [100]:
from selenium import webdriver
# from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [56]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# print(chrome_options)

In [123]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [126]:
url = "https://iis.ncog.gov.in/parks/exploreParkReport"
driver.get(url)

# wait = WebDriverWait(driver, 10)
# wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Next']"))).click()
# wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='i8']"))).click()


In [228]:
# select = Select(driver.find_element(By.XPATH, '/html/body/section/div/div[3]/div/div/div/label/select'))
# select = Select(driver.find_element(By.NAME, 'selection_length'))
select = Select(driver.find_element(By.XPATH, "//select[@name='selection_length']"))

select.select_by_visible_text('All') 
# print(select)

In [304]:
# table_head = driver.find_element(By.XPATH, '//section/div/div[3]/div/div[2]/div/div/table').get_attribute('outerHTML')
# table = wait.until(EC.visibility_of_element_located((By.XPATH, '//html/body/section/')))
# table_head = driver.find_element(By.CLASS_NAME, 'dataTables_scrollHeadInner').get_attribute('outerHTML')
table_head = driver.find_element(By.XPATH, "//div[@class='dataTables_scrollHeadInner']/table").get_attribute('outerHTML')

df = pd.read_html(table_head)[0]
# print(df.head())

In [305]:
# table_body = driver.find_element(By.XPATH, '//section/div/div[3]/div/div[2]/div[2]/table').get_attribute('outerHTML')
# table_body = driver.find_element(By.CLASS_NAME, 'dataTables_scrollBody').get_attribute('outerHTML')
table_body = driver.find_element(By.XPATH, "//div[@class='dataTables_scrollBody']/table").get_attribute('outerHTML')

df_body = pd.read_html(table_body)[0]
# print(df_body)

In [285]:
from datetime import date
import datetime
import numpy
import math

In [94]:
date.today().year

2022

In [306]:
df_body.columns = df.columns
df = df_body
df.drop(['SrNO', 'ViewOnMap'], axis=1, inplace=True)
df.insert(0, 'State', df.pop('State'))
df.insert(1, 'Year', date.today().year)
# df.insert(len(df.columns), 'asondate', date.today())
df.fillna('', inplace=True)
print(df)

             State  Year  Park ID  ... Park Type                Sector Update Date
0        MEGHALAYA  2022     6868  ...    Public                 Mixed  2022-02-21
1     CHHATTISGARH  2022     3194  ...    Public  Electronics Hardware  2022-02-21
2           KERALA  2022     7853  ...    Public         Capital Goods            
3           Odisha  2022     5944  ...    Public  Electronics Hardware  2022-02-21
4      WEST BENGAL  2022     3508  ...    Public  Electronics Hardware  2022-02-21
...            ...   ...      ...  ...       ...                   ...         ...
4560       GUJARAT  2022      421  ...    Public                 Mixed  2020-07-08
4561     KARNATAKA  2022     7097  ...    Public                 Mixed  2020-11-11
4562        Odisha  2022     5938  ...    Public                 Mixed            
4563     KARNATAKA  2022     7037  ...    Public  Electronics Hardware  2021-03-03
4564       GUJARAT  2022      379  ...    Public                 Mixed  2020-07-02

[45

In [301]:
def get_df_with_datatypes(df, ignore_cols = "Year"):
    """
    Function takes the pandas dataframe and ignore_cols as input parameters.
    Returns the final data frame by appending the SQL data types in the first row 
    for each column in the data frame based on the python native datatypes.
    "ignore_cols": multiple column names can be passed by comma separated, these column will get the standard data type of 
    VARCHAR irrespective of the python native data type. 
    (this parameter is useful for the columns which has integer values but should be treated as string values, ex: Year)

    """
    data_types = []
    ignore_cols = ignore_cols.lower().split(",")    # convert all ignore cols to lowercase
    
    # Add the "year" column to ignore_cols if not passed 
    if 'year' not in ignore_cols:   
        ignore_cols.append('year')
    # print(df.columns)
    
    # define the data types for each column of the dataframe
    for column in df.columns:
        py_type = type(df[column].values[1])    # get the python native datatype
        # print(py_type)
        
        # if the column is specified in ignore_cols, then a standard VARCHAR is defined as SQL datatype
        if column.lower() in ignore_cols: 
            data_types.append("VARCHAR(30)")
            continue
        # if the column is of string type, then VARCHAR length is fixed by 3 times the max length of that column
        elif py_type==str: 
            # print("Column Name: ",column, "\t\t\tMax Length: ", len(max(df[column].values, key=len)))
            max_len = math.ceil(3*len(max(df[column].values, key=len)))
            max_len = max_len - (max_len % 10) if max_len>20 else 20
            data_types.append("VARCHAR("+str(max_len)+")")
        # if the column is of integer type, then INT
        elif py_type==numpy.int64:
            data_types.append("INT")
        # if the column is of float type, then DECIMAL
        elif py_type==numpy.float64:
            data_types.append("DECIMAL")
        # if the column is of date type, then DATE
        elif py_type==datetime.date:
            data_types.append("DATE")
        # if the column type is not recognized as any of the above , then VARCHAR with fixed length is specified
        else:
            data_types.append("VARCHAR(50)")

    data_types = pd.DataFrame([data_types], index=[0], columns=df.columns.values)
    df = pd.concat([data_types, df]).reset_index(drop=True)
    return df

In [308]:
df = get_df_with_datatypes(df, ignore_cols = "Park ID")

(df)

,State,Year,Park ID,User,User Level,District,Park Name,Industrial Park Type,Total Land Area (ha.),Land Available (ha.),Plug And Play,Park Type,Sector,Update Date
0,VARCHAR(50),VARCHAR(30),VARCHAR(30),VARCHAR(60),VARCHAR(30),VARCHAR(250),VARCHAR(260),VARCHAR(120),VARCHAR(50),VARCHAR(50),VARCHAR(30),VARCHAR(40),VARCHAR(100),VARCHAR(30)
1,VARCHAR(50),VARCHAR(30),VARCHAR(30),VARCHAR(60),VARCHAR(20),VARCHAR(250),VARCHAR(260),VARCHAR(120),DECIMAL,DECIMAL,VARCHAR(20),VARCHAR(40),VARCHAR(100),VARCHAR(30)
2,MEGHALAYA,2022,6868,Meghalaya,State,Ribhoi,Export Promotion Industrial Park (EPIP),Industrial Park,125.45,0.0,No,Public,Mixed,2022-02-21
3,CHHATTISGARH,2022,3194,doicg_iis,State,Raipur,"Electronic Manufacturing Cluster, Naya Raipur",Industrial Area,48.57,22.84,No,Public,Electronics Hardware,2022-02-21
4,KERALA,2022,7853,keralasidco,State,Thrissur,"Kerala Sidco Mini Industrial Estate, Kattoor",Industrial Park,0.0,0.08,,Public,Capital Goods,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562,GUJARAT,2022,421,Gujarat,State,Ahmadabad,Bhagapura I,Industrial Estate,285.49,100.0,Yes,Public,Mixed,2020-07-08
4563,KARNATAKA,2022,7097,KIADB,State,Bagalkot,Navanagar Agrotech Park,Industrial Area,89.59,0.0,,Public,Mixed,2020-11-11
4564,Odisha,2022,5938,Odisha,State,Bolangir,I.E Bolangir Zone-A,Industrial Estate,5.45,0.28,,Public,Mixed,
4565,KARNATAKA,2022,7037,KIADB,State,Bangalore Rural,Bengaluru Hardware Park,Industrial Area,539.0,0.0,,Public,Electronics Hardware,2021-03-03


In [ ]:
# data_types = pd.DataFrame([data_types], index=[0], columns=df.columns.values)
# df = pd.concat([data_types, df]).reset_index(drop=True)
# df

In [297]:
df.to_csv("MOFPI_IILB_Industrial Parks.csv", index=False, encoding='utf-8')

# Census

In [ ]:
!wget --header="Host: doc-0k-c4-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/" --header="Cookie: AUTH_6q6v5cpktfsbr56mgd97qaku8fu7piaj_nonce=88hko6iqrt212" --header="Connection: keep-alive" "https://doc-0k-c4-docs.googleusercontent.com/docs/securesc/nka8he3o66l2nqa1l22al91eq1deois1/hb8236d683ife5okc3vitoaervfv13u4/1644495450000/11589316532753017289/06457702143635460075/1qgf4tfU90SKrldNHK9tr8XNnJiocChjq?e=download&authuser=0&nonce=88hko6iqrt212&user=06457702143635460075&hash=amq4e1at86924m479lkmu5pau48cq4jf" -c -O 'Census_Village_new.rar'

In [12]:
path_to_zip_file = "/content/Census States2.zip"
directory_to_extract_to = "censusdata"

In [13]:
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [14]:
path_to_rar_file = "/content/Census_Village_new.rar"

In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive(path_to_rar_file, outdir=directory_to_extract_to)

In [17]:
states = os.listdir("censusdata")

In [19]:
print(len(states))

35


In [20]:
walk_dir = "censusdata"

In [24]:
dest_dir = "censusconsolidate"

In [86]:
state_dict = {}
j = 0
for root, subdirs, files in os.walk(walk_dir):
    j = j+1
    if j<=36:
        continue
    # print('--\nroot = ' + root, subdirs)
    # list_file_path = os.path.join(root, 'my-directory-list.txt')
    # print('list_file_path = ' + list_file_path)
    if len(subdirs)<=0:
        print (root, subdirs, files)
        # break
        state = root.split("/")[1]
        state_dict[state] = {}
        k = 0
        for file in files:
            k = k+1
            state_dict[state][k] = pd.read_excel(os.path.join(root, file))

        # state_df = pd.DataFrame()
        for key, val in state_dict[state].items():
            if key==1: 
                state_df = val
                # print(type(state_df))
            if key>1:
                state_df = state_df.append(val, ignore_index=True)
                # print(type(state_df))

        # print(state_df)
        state_df.to_excel("censusconsolidate/"+state+".xlsx", index=False)
        if j==38:
            break


censusdata/kerala [] ['PCA CDB-3208-F-Census.xlsx', 'PCA CDB-3202-F-Census.xlsx', 'PCA CDB-3213-F-Census.xlsx', 'PCA CDB-3211-F-Census.xlsx', 'PCA CDB-3201-F-Census.xlsx', 'PCA CDB-3207-F-Census.xlsx', 'PCA CDB-3214-F-Census.xlsx', 'PCA CDB-3210-F-Census.xlsx', 'PCA CDB-3203-F-Census.xlsx', 'PCA CDB-3212-F-Census.xlsx', 'PCA CDB-3205-F-Census.xlsx', 'PCA CDB-3206-F-Census.xlsx', 'PCA CDB-3204-F-Census.xlsx', 'PCA CDB-3209-F-Census.xlsx']
censusdata/ap [] ['PCA CDB-2812-F-Census.xlsx', 'PCA CDB-2820-F-Census.xlsx', 'PCA CDB-2806-F-Census.xlsx', 'PCA CDB-2804-F-Census.xlsx', 'PCA CDB-2807-F-Census.xlsx', 'PCA CDB-2818-F-Census.xlsx', 'PCA CDB-2816-F-Census.xlsx', 'PCA CDB-2815-F-Census.xlsx', 'PCA CDB-2801-F-Census.xlsx', 'PCA CDB-2808-F-Census.xlsx', 'PCA CDB-2813-F-Census.xlsx', 'PCA CDB-2823-F-Census.xlsx', 'PCA CDB-2802-F-Census.xlsx', 'PCA CDB-2814-F-Census.xlsx', 'PCA CDB-2809-F-Census.xlsx', 'PCA CDB-2817-F-Census.xlsx', 'PCA CDB-2805-F-Census.xlsx', 'PCA CDB-2811-F-Census.xlsx', 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
!cp "/content/census_final11_20.xlsx" "gdrive/My Drive/census_final11_20.xlsx"

In [89]:
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

zipfolder('census_state_consol', 'censusconsolidate') #insert your variables here
# sys.exit()

In [7]:
walk_dir1 = "/content/censusconsolidate"
final_census = {}
k = 0
for root, subdirs, files in os.walk(walk_dir1):
    # print('--\nroot = ' + root, subdirs, files)
    for file in files:
        k = k+1
        print(k, file)
        if k==1:
            final_census = pd.read_excel(os.path.join(root, file))
        if k>1:
            final_census = final_census.append(pd.read_excel(os.path.join(root, file)), ignore_index=True)
        if k==10:
            break

    final_census.to_excel("census_final1_10.xlsx", index=False)


1 Maharashtra.xlsx
2 ap.xlsx
3 Uttar Pradesh.xlsx
4 haryana.xlsx
5 jharkand.xlsx
6 Sikkim.xlsx
7 dadra.xlsx
8 Tamil Nadu.xlsx
9 arunachal.xlsx
10 Meghalaya.xlsx


In [15]:
walk_dir1 = "/content/censusconsolidate"
# final_census = {}
k = 0
for root, subdirs, files in os.walk(walk_dir1):
    # print('--\nroot = ' + root, subdirs, files)
    for file in files:
        k = k+1
        print(k, file)
        if k <= 10:
            continue
        if k==11:
            final_census = pd.read_excel(os.path.join(root, file))
        if k>11:
            final_census = final_census.append(pd.read_excel(os.path.join(root, file)), ignore_index=True)
        if k==20:
            break

    final_census.to_excel("census_final11_20.xlsx", index=False)


1 Maharashtra.xlsx
2 ap.xlsx
3 Uttar Pradesh.xlsx
4 haryana.xlsx
5 jharkand.xlsx
6 Sikkim.xlsx
7 dadra.xlsx
8 Tamil Nadu.xlsx
9 arunachal.xlsx
10 Meghalaya.xlsx
11 Manipur.xlsx
12 Nagaland.xlsx
13 chandigarh.xlsx
14 chattiss.xlsx
15 andaman.xlsx
16 jammu.xlsx
17 Rajasthan.xlsx
18 Mizoram.xlsx
19 Uttarakhand.xlsx
20 himachal.xlsx


In [11]:
walk_dir1 = "/content/censusconsolidate"
# final_census = {}
k = 0
for root, subdirs, files in os.walk(walk_dir1):
    # print('--\nroot = ' + root, subdirs, files)
    for file in files:
        k = k+1
        print(k, file)
        if k <= 20:
            continue
        if k==21:
            final_census = pd.read_excel(os.path.join(root, file))
        if k>21:
            final_census = final_census.append(pd.read_excel(os.path.join(root, file)), ignore_index=True)
        if k==30:
            break

    final_census.to_excel("census_final21_30.xlsx", index=False)


1 Maharashtra.xlsx
2 ap.xlsx
3 Uttar Pradesh.xlsx
4 haryana.xlsx
5 jharkand.xlsx
6 Sikkim.xlsx
7 dadra.xlsx
8 Tamil Nadu.xlsx
9 arunachal.xlsx
10 Meghalaya.xlsx
11 Manipur.xlsx
12 Nagaland.xlsx
13 chandigarh.xlsx
14 chattiss.xlsx
15 andaman.xlsx
16 jammu.xlsx
17 Rajasthan.xlsx
18 Mizoram.xlsx
19 Uttarakhand.xlsx
20 himachal.xlsx
21 Odisha.xlsx
22 assam.xlsx
23 West Bengal.xlsx
24 Lakshadweep.xlsx
25 Punjab.xlsx
26 karnataka.xlsx
27 Tripura.xlsx
28 Bihar.xlsx
29 nct of delhi.xlsx
30 gujarat.xlsx


In [13]:
walk_dir1 = "/content/censusconsolidate"
# final_census = {}
k = 0
for root, subdirs, files in os.walk(walk_dir1):
    # print('--\nroot = ' + root, subdirs, files)
    for file in files:
        k = k+1
        print(k, file)
        if k <= 30:
            continue
        if k==31:
            final_census = pd.read_excel(os.path.join(root, file))
        if k>31:
            final_census = final_census.append(pd.read_excel(os.path.join(root, file)), ignore_index=True)
        if k==40:
            break

    final_census.to_excel("census_final31_40.xlsx", index=False)


1 Maharashtra.xlsx
2 ap.xlsx
3 Uttar Pradesh.xlsx
4 haryana.xlsx
5 jharkand.xlsx
6 Sikkim.xlsx
7 dadra.xlsx
8 Tamil Nadu.xlsx
9 arunachal.xlsx
10 Meghalaya.xlsx
11 Manipur.xlsx
12 Nagaland.xlsx
13 chandigarh.xlsx
14 chattiss.xlsx
15 andaman.xlsx
16 jammu.xlsx
17 Rajasthan.xlsx
18 Mizoram.xlsx
19 Uttarakhand.xlsx
20 himachal.xlsx
21 Odisha.xlsx
22 assam.xlsx
23 West Bengal.xlsx
24 Lakshadweep.xlsx
25 Punjab.xlsx
26 karnataka.xlsx
27 Tripura.xlsx
28 Bihar.xlsx
29 nct of delhi.xlsx
30 gujarat.xlsx
31 kerala.xlsx
32 Madhya Pradesh.xlsx
33 goa.xlsx
34 Puducherry.xlsx
35 daman.xlsx


In [32]:
df21_30 = pd.read_excel("/content/census_final21_30.xlsx")

In [33]:
df21_30.head()
df21_30.shape

(231920, 95)

In [34]:
df21_30.to_csv('census_final21_30.csv', index = False)

In [35]:
df_temp = pd.read_csv("census_final21_30.csv")
df_temp.head()

,State,District,DT Name,CD Block,Town/Village,Ward,EB,Level,Name,TRU,No_HH,TOT_P,TOT_M,TOT_F,P_06,M_06,F_06,P_SC,M_SC,F_SC,P_ST,M_ST,F_ST,P_LIT,M_LIT,F_LIT,P_ILL,M_ILL,F_ILL,TOT_WORK_P,TOT_WORK_M,TOT_WORK_F,MAINWORK_P,MAINWORK_M,MAINWORK_F,MAIN_CL_P,MAIN_CL_M,MAIN_CL_F,MAIN_AL_P,MAIN_AL_M,...,MARG_AL_F,MARG_HH_P,MARG_HH_M,MARG_HH_F,MARG_OT_P,MARG_OT_M,MARG_OT_F,MARGWORK_3_6_P,MARGWORK_3_6_M,MARGWORK_3_6_F,MARG_CL_3_6_P,MARG_CL_3_6_M,MARG_CL_3_6_F,MARG_AL_3_6_P,MARG_AL_3_6_M,MARG_AL_3_6_F,MARG_HH_3_6_P,MARG_HH_3_6_M,MARG_HH_3_6_F,MARG_OT_3_6_P,MARG_OT_3_6_M,MARG_OT_3_6_F,MARGWORK_0_3_P,MARGWORK_0_3_M,MARGWORK_0_3_F,MARG_CL_0_3_P,MARG_CL_0_3_M,MARG_CL_0_3_F,MARG_AL_0_3_P,MARG_AL_0_3_M,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
0,21,393,Balangir,241,0,0,0,CD BLOCK,Agalpur,Total,25513,96342,48602,47740,10941,5598,5343,17547,8839,8708,12268,6232,6036,59242,34246,24996,37100,14356,22744,43996,28186,15810,23096,19763,3333,8382,7957,425,8261,6495,...,9721,1291,460,831,2141,1224,917,16417,6754,9663,1648,942,706,12068,4489,7579,1031,360,671,1670,963,707,4483,1669,2814,415,113,302,3337,1195,2142,260,100,160,471,261,210,52346,20416,31930
1,21,393,Balangir,241,0,0,0,CD BLOCK,Agalpur,Rural,25513,96342,48602,47740,10941,5598,5343,17547,8839,8708,12268,6232,6036,59242,34246,24996,37100,14356,22744,43996,28186,15810,23096,19763,3333,8382,7957,425,8261,6495,...,9721,1291,460,831,2141,1224,917,16417,6754,9663,1648,942,706,12068,4489,7579,1031,360,671,1670,963,707,4483,1669,2814,415,113,302,3337,1195,2142,260,100,160,471,261,210,52346,20416,31930
2,21,393,Balangir,241,0,0,0,CD BLOCK,Agalpur,Urban,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,21,393,Balangir,241,419815,0,0,VILLAGE,Mundapali,Rural,170,654,295,359,61,30,31,126,61,65,34,17,17,381,189,192,273,106,167,393,175,218,143,129,14,93,92,1,11,8,...,201,0,0,0,18,15,3,250,46,204,2,2,0,230,29,201,0,0,0,18,15,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,261,120,141
4,21,393,Balangir,241,419816,0,0,VILLAGE,Gondpali,Rural,83,325,165,160,45,26,19,22,14,8,235,115,120,188,111,77,137,54,83,208,102,106,108,92,16,39,39,0,61,48,...,90,0,0,0,0,0,0,96,10,86,0,0,0,96,10,86,0,0,0,0,0,0,4,0,4,0,0,0,4,0,4,0,0,0,0,0,0,117,63,54


In [41]:
!cp "/content/census_final.csv" "gdrive/My Drive/census_final.csv"

In [37]:
df10 = pd.read_csv("census_final1_10.csv")
df20 = pd.read_csv("census_final11_20.csv")
df30 = pd.read_csv("census_final21_30.csv")
df40 = pd.read_csv("census_final31_40.csv")

census_final = pd.concat([df10, df20, df30, df40], ignore_index=True)

In [38]:
census_final.to_csv('census_final.csv', index = False)

In [40]:
census_final.head()

,State,District,DT Name,CD Block,Town/Village,Ward,EB,Level,Name,TRU,No_HH,TOT_P,TOT_M,TOT_F,P_06,M_06,F_06,P_SC,M_SC,F_SC,P_ST,M_ST,F_ST,P_LIT,M_LIT,F_LIT,P_ILL,M_ILL,F_ILL,TOT_WORK_P,TOT_WORK_M,TOT_WORK_F,MAINWORK_P,MAINWORK_M,MAINWORK_F,MAIN_CL_P,MAIN_CL_M,MAIN_CL_F,MAIN_AL_P,MAIN_AL_M,...,MARG_HH_P,MARG_HH_M,MARG_HH_F,MARG_OT_P,MARG_OT_M,MARG_OT_F,MARGWORK_3_6_P,MARGWORK_3_6_M,MARGWORK_3_6_F,MARG_CL_3_6_P,MARG_CL_3_6_M,MARG_CL_3_6_F,MARG_AL_3_6_P,MARG_AL_3_6_M,MARG_AL_3_6_F,MARG_HH_3_6_P,MARG_HH_3_6_M,MARG_HH_3_6_F,MARG_OT_3_6_P,MARG_OT_3_6_M,MARG_OT_3_6_F,MARGWORK_0_3_P,MARGWORK_0_3_M,MARGWORK_0_3_F,MARG_CL_0_3_P,MARG_CL_0_3_M,MARG_CL_0_3_F,MARG_AL_0_3_P,MARG_AL_0_3_M,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F,Area
0,27,507,Gondiya,95,0,0,0,CD BLOCK,Tirora,Total,33865,151073,76216,74857,16198,8337,7861,17385,8638,8747,13075,6608,6467,117908,63934,53974,33165,12282,20883,79145,44750,34395,53793,33716,20077,21008,13639,7369,21709,12433,...,1386,456,930,4070,2584,1486,21061,9633,11428,4373,2081,2292,12447,5114,7333,1139,395,744,3102,2043,1059,4291,1401,2890,988,242,746,2088,557,1531,247,61,186,968,541,427,71928,31466,40462,NaN
1,27,507,Gondiya,95,0,0,0,CD BLOCK,Tirora,Rural,33865,151073,76216,74857,16198,8337,7861,17385,8638,8747,13075,6608,6467,117908,63934,53974,33165,12282,20883,79145,44750,34395,53793,33716,20077,21008,13639,7369,21709,12433,...,1386,456,930,4070,2584,1486,21061,9633,11428,4373,2081,2292,12447,5114,7333,1139,395,744,3102,2043,1059,4291,1401,2890,988,242,746,2088,557,1531,247,61,186,968,541,427,71928,31466,40462,NaN
2,27,507,Gondiya,95,0,0,0,CD BLOCK,Tirora,Urban,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
3,27,507,Gondiya,95,537518,0,0,VILLAGE,Pipariya,Rural,226,1005,478,527,123,59,64,53,26,27,0,0,0,740,399,341,265,79,186,592,311,281,546,292,254,198,138,60,275,90,...,1,1,0,0,0,0,33,13,20,1,1,0,32,12,20,0,0,0,0,0,0,13,6,7,0,0,0,12,5,7,1,1,0,0,0,0,413,167,246,NaN
4,27,507,Gondiya,95,537519,0,0,VILLAGE,Sawara,Rural,146,687,363,324,58,34,24,0,0,0,41,20,21,533,302,231,154,61,93,390,212,178,184,139,45,76,65,11,105,74,...,2,1,1,0,0,0,145,43,102,24,5,19,119,37,82,2,1,1,0,0,0,61,30,31,7,2,5,54,28,26,0,0,0,0,0,0,297,151,146,NaN


In [1]:
final_census.items()

NameError: ignored

In [3]:
census_df = pd.read_csv("/content/gdrive/MyDrive/census_final.csv")
census_df.shape

(665310, 96)

In [16]:
census_df[~census_df['Area'].isna()]

,State,District,DT Name,CD Block,Town/Village,Ward,EB,Level,Name,TRU,No_HH,TOT_P,TOT_M,TOT_F,P_06,M_06,F_06,P_SC,M_SC,F_SC,P_ST,M_ST,F_ST,P_LIT,M_LIT,F_LIT,P_ILL,M_ILL,F_ILL,TOT_WORK_P,TOT_WORK_M,TOT_WORK_F,MAINWORK_P,MAINWORK_M,MAINWORK_F,MAIN_CL_P,MAIN_CL_M,MAIN_CL_F,MAIN_AL_P,MAIN_AL_M,...,MARG_HH_P,MARG_HH_M,MARG_HH_F,MARG_OT_P,MARG_OT_M,MARG_OT_F,MARGWORK_3_6_P,MARGWORK_3_6_M,MARGWORK_3_6_F,MARG_CL_3_6_P,MARG_CL_3_6_M,MARG_CL_3_6_F,MARG_AL_3_6_P,MARG_AL_3_6_M,MARG_AL_3_6_F,MARG_HH_3_6_P,MARG_HH_3_6_M,MARG_HH_3_6_F,MARG_OT_3_6_P,MARG_OT_3_6_M,MARG_OT_3_6_F,MARGWORK_0_3_P,MARGWORK_0_3_M,MARGWORK_0_3_F,MARG_CL_0_3_P,MARG_CL_0_3_M,MARG_CL_0_3_F,MARG_AL_0_3_P,MARG_AL_0_3_M,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F,Area
260981,13,265,NaN,3,0,0,0,CD BLOCK,Dhansiripar,Total,1005,6673,4973,1700,583,322,261,0,0,0,118,70,48,5783,4539,1244,890,434,456,4380,4142,238,4276,4091,185,59,50,9,38,24,...,1,1,0,95,49,46,99,49,50,2,0,2,5,1,4,0,0,0,92,48,44,5,2,3,0,0,0,1,0,1,1,1,0,3,1,2,2293,831,1462,0.0
260982,13,265,NaN,3,0,0,0,CD BLOCK,Dhansiripar,Rural,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
260983,13,265,NaN,3,0,0,0,CD BLOCK,Dhansiripar,Urban,1005,6673,4973,1700,583,322,261,0,0,0,118,70,48,5783,4539,1244,890,434,456,4380,4142,238,4276,4091,185,59,50,9,38,24,...,1,1,0,95,49,46,99,49,50,2,0,2,5,1,4,0,0,0,92,48,44,5,2,3,0,0,0,1,0,1,1,1,0,3,1,2,2293,831,1462,0.0
260984,13,265,NaN,3,267668,0,0,TOWN,Rangapahar (CT),Urban,1005,6673,4973,1700,583,322,261,0,0,0,118,70,48,5783,4539,1244,890,434,456,4380,4142,238,4276,4091,185,59,50,9,38,24,...,1,1,0,95,49,46,99,49,50,2,0,2,5,1,4,0,0,0,92,48,44,5,2,3,0,0,0,1,0,1,1,1,0,3,1,2,2293,831,1462,0.0
260985,13,265,NaN,4,0,0,0,CD BLOCK,Medziphema,Total,7050,33739,17184,16555,4156,2101,2055,0,0,0,19968,9840,10128,26122,13602,12520,7617,3582,4035,11867,7917,3950,10028,7193,2835,169,75,94,104,60,...,299,68,231,1301,572,729,1470,595,875,107,38,69,51,26,25,255,58,197,1057,473,584,369,129,240,64,15,49,17,5,12,44,10,34,244,99,145,21872,9267,12605,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261938,13,270,NaN,49,268218,0,0,VILLAGE,Rengmapani,Rural,131,627,303,324,154,77,77,0,0,0,550,259,291,362,179,183,265,124,141,239,138,101,234,135,99,196,110,86,17,9,...,0,0,0,0,0,0,4,3,1,1,1,0,3,2,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,388,165,223,0.0
261939,13,270,NaN,49,268219,0,0,VILLAGE,Tsosinyu,Rural,137,696,367,329,141,80,61,0,0,0,696,367,329,322,178,144,374,189,185,368,179,189,359,178,181,340,162,178,0,0,...,0,0,0,3,0,3,9,1,8,6,1,5,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,328,188,140,0.0
261940,13,270,NaN,49,268220,0,0,VILLAGE,New Sendenyu,Rural,140,730,394,336,141,70,71,0,0,0,715,384,331,564,309,255,166,85,81,384,209,175,379,204,175,351,186,165,0,0,...,0,0,0,5,5,0,5,5,0,0,0,0,0,0,0,0,0,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,346,185,161,0.0
261941,13,270,NaN,49,268221,0,0,VILLAGE,Sendenyu,Rural,498,2548,1299,1249,515,257,258,0,0,0,2506,1279,1227,1875,972,903,673,327,346,1099,535,564,1082,530,552,1042,509,533,3,1,...,0,0,0,12,3,9,11,4,7,3,2,1,0,0,0,0,0,0,8,2,6,6,1,5,2,0,2,0,0,0,0,0,0,4,1,3,1449,764,685,0.0


# PDF Extraction - Misc tools

In [3]:
!pip install pdfminer.six




     |████████████████████████████████| 5.6 MB 8.2 MB/s 
     |████████████████████████████████| 3.6 MB 63.7 MB/s 


In [6]:
!pdf2txt.py -o dl_factsheet.html -t html DL_FactSheet.pdf

# Misc

In [1]:
import pandas as pd

In [9]:
xls = pd.ExcelFile("NDAP_21022022.xlsx")
xls.sheet_names
for sheetName in xls.sheet_names:
    df = pd.read_excel("NDAP_21022022.xlsx", sheet_name=sheetName)
    df.to_csv(sheetName+".csv")

In [15]:
df = pd.read_csv("/content/ODOP-FME.csv")

df

,Unnamed: 0,ODOP-FME,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,0,S.No.,State,District Name,PM-FME Scheme Product (The scheme is SHG based)
1,1,1,ANDAMAN & NICOBAR ISLAND,Middle & North Andaman,Coconut Based Products
2,2,2,ANDAMAN & NICOBAR ISLAND,Nicobar,Coconut Based Products
3,3,3,ANDAMAN & NICOBAR ISLAND,South Andaman,Marine Fish Products
4,4,4,ANDHRA PRADESH,Ananthapur,Groundnut Products
...,...,...,...,...,...
702,702,702,UTTARAKHAND,Pithoragarh,Turmeric based product
703,703,703,UTTARAKHAND,Rudraprayag,Amaranthas based products (Laddu etc.)
704,704,704,UTTARAKHAND,Tehri,"Ginger based product (Dry Ginger, Candy, Pickl..."
705,705,705,UTTARAKHAND,Udham Singh Nagar,Mango based product
